In [12]:
import redis
import multiprocessing.pool

In [13]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [16]:
r.hlen("stock_tick_current_day")

8301427

In [3]:
r.hset("sample", 'ss', 123)

1

In [4]:
r.hget("sample", 'ss')

'123'

In [ ]:
fetcher = quotation.Sina()

In [ ]:
a = fetcher.market_snapshot()

In [ ]:
def get_key_item(k , v):
    key = f"""{k[0]}#{k[1]}"""
    item = ",".join([str(v[i]) for i in v])
    return key, item

In [ ]:
kv_list = [get_key_item(i, a[i]) for i in a]

In [62]:
def set2redis(kv):
    try:
        k = kv[0]
        v = kv[1]
        r.hset('sample', k, v)
    except Exception as e:
        return None

In [ ]:
%%timeit
pool = multiprocessing.pool.ThreadPool(2)
try:
    res = pool.map(set2redis, kv_list)
finally:
    pool.close()

In [7]:
import pandas as pd

In [8]:
df = pd.read_parquet("/mnt/stock_data/tmp_dask/2020-05-22.parquet")

In [10]:
12131140/15483320

0.7834973377802693

In [9]:
df.shape

(15483320, 25)

In [78]:
df = df.reset_index()

In [79]:
def series2kv(r):
    try:
        res = (f"""{r['code']}#{r['time']}""", ",".join([str(i) for i in r.to_list()[2:]]))
        return res
    except Exception as e:
        return None

In [ ]:
kv_list = df.apply(lambda r: series2kv(r), axis=1).to_list()

In [84]:
%%time
pool = multiprocessing.pool.ThreadPool(2)
try:
    res = pool.map(set2redis, kv_list)
finally:
    pool.close()

CPU times: user 10min 44s, sys: 4min 18s, total: 15min 2s
Wall time: 10min 46s


In [90]:
# %%timeit
pipe = r.pipeline()

In [97]:
%%timeit
for kv in kv_list[-5000:]:
    pipe.hset('sample', kv[0], kv[1])
a = pipe.execute()
pipe.reset()

55.2 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [96]:
%%timeit
pool = multiprocessing.pool.ThreadPool(1)
try:
    res = pool.map(set2redis, kv_list[-5000:])
finally:
    pool.close()

311 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
[r.hget('sample' ,i[0]) for i in kv_list[-10:]]

In [ ]:
[i[1] for i in kv_list[-10:]]

In [98]:
a = r.hgetall('sample')

In [99]:
len(a)/5000/20/60

2.580624